**Общая информация**

**Срок сдачи:** 18 октября 2017, 06:00 <br\>
**Штраф за опоздание:** -2 балла после 06:00 18 октября, -4 балла после 06:00 25 октября, -6 баллов после 06:00 1 ноября

При отправлении ДЗ указывайте фамилию в названии файла
Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush

Необходимо в slack создать таск в приватный чат:
/todo Фамилия Имя ссылка на гитхаб @alkhamush
Пример:
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw1.ipynb @alkhamush
Дополнительно нужно просто скинуть ссылку в slack в личный чат

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (2 баллов)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. Самостоятельно разберитесь, как считается score из KNeighborsClassifier и реализуйте аналог в своём классе. Score не должен уступать значению KNN из sklearn

###### Задание 2 (2 балла)
Добиться скорости работы на fit,  predict и predict_proba сравнимой со sklearn для iris и mnist
Для этого используем numpy

###### Задание 3 (2 балла)
Для iris найдите такой параметр n_neighbors, при котором выдаётся наилучший score. Нарисуйте график зависимости score от n_neighbors

###### Задание 4 (3 балла)
Добавить algorithm='kd_tree' в реализацию KNN (использовать KDTree из sklearn.neighbors). Значение n_neighbors нужно взять из задания 3. Добиться скорости работы на fit,  predict и predict_proba сравнимой со sklearn для iris и mnist
Для этого используем numpy. Score не должен уступать значению KNN из sklearn

###### Задание 5 (1 балла)
Описать для чего нужны следующие библиотеки/классы/функции (список будет ниже)

**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw1.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять структуру кода и текст. В противном случае -1 балл


In [187]:
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn import datasets
from sklearn.base import ClassifierMixin
from sklearn.datasets import fetch_mldata
from sklearn.neighbors.base import NeighborsBase, KNeighborsMixin, SupervisedIntegerMixin 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
%load_ext pycodestyle_magic

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


In [188]:
#%%pycodestyle

class MyKNeighborsClassifier(NeighborsBase,
                             KNeighborsMixin,
                             SupervisedIntegerMixin,
                             ClassifierMixin):

    def __init__(self, n_neighbors, algorithm='brute'):
        self.n_neighbors = n_neighbors
        self.algorithm = algorithm

    def fit(self, X, y):
        self.X = X
        self.labels = y
        self.number_of_classes = len(set(self.labels))
        
    def _get_class_matrix(self, X):
        #print('start', time.ctime())
        v = np.apply_along_axis(lambda xi: self.X-xi, 1, X)
        #print('diff done', time.ctime())
        v = np.linalg.norm(v, axis=2)
        #print('norm done', time.ctime())
        r = v.argsort()[:,:self.n_neighbors]
        r = np.argpartition(v, self.n_neighbors)[:, :self.n_neighbors]
        #print('argsort done', time.ctime())
        u, indices = np.unique(self.labels[r], return_inverse=True)
        #print('uniq done', time.ctime())
        return np.apply_along_axis(np.bincount, 1, indices.reshape(r.shape), None, np.max(indices) + 1), u
        
    def predict(self, X):
        v, u = self._get_class_matrix(X)
        return u[np.argmax(v, axis=1)]

    def predict_proba(self, X):
        v, u = self._get_class_matrix(X)
        return v / np.sum(v, axis=1)[:,None]

    def score(self, X, y):
        return 1 - np.count_nonzero(self.predict(X)- y) / y.shape[0]

**IRIS**

In [189]:
iris = datasets.load_iris()

In [190]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [191]:
clf = KNeighborsClassifier(n_neighbors=2)
my_clf = MyKNeighborsClassifier(n_neighbors=2)

In [192]:
r =np.array([[1, 1, 1], [1, 0, 1], [0, 0, 1]])
u, indices = np.unique(r, return_inverse=True)
u[np.argmax(np.apply_along_axis(np.bincount, axis, indices.reshape(r.shape),
                                        None, np.max(indices) + 1), axis=1)]

array([1, 1, 0])

In [193]:
X_test

array([[7.7, 2.6, 6.9, 2.3],
       [4.3, 3. , 1.1, 0.1],
       [4.6, 3.4, 1.4, 0.3],
       [4.9, 3.1, 1.5, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [6.6, 2.9, 4.6, 1.3],
       [6.5, 3. , 5.5, 1.8],
       [5. , 3.5, 1.3, 0.3],
       [5.5, 2.4, 3.8, 1.1],
       [5.6, 2.5, 3.9, 1.1],
       [7.9, 3.8, 6.4, 2. ],
       [5.8, 2.7, 4.1, 1. ],
       [5.7, 3. , 4.2, 1.2],
       [6.5, 3. , 5.2, 2. ],
       [7.4, 2.8, 6.1, 1.9]])

In [194]:
%time clf.fit(X_train, y_train)

CPU times: user 1.57 ms, sys: 1.94 ms, total: 3.51 ms
Wall time: 3.24 ms


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=2, p=2,
           weights='uniform')

In [195]:
%time my_clf.fit(X_train, y_train)

CPU times: user 37 µs, sys: 6 µs, total: 43 µs
Wall time: 47.2 µs


In [196]:
%time clf.predict(X_test)

CPU times: user 1.94 ms, sys: 3.51 ms, total: 5.45 ms
Wall time: 5.76 ms


array([2, 0, 0, 0, 0, 1, 2, 0, 1, 1, 2, 1, 1, 2, 2])

In [197]:
%time my_clf.predict(X_test)

CPU times: user 1.94 ms, sys: 2.49 ms, total: 4.43 ms
Wall time: 9.45 ms


array([2, 0, 0, 0, 0, 1, 2, 0, 1, 1, 2, 1, 1, 2, 2])

In [198]:
%time clf.predict_proba(X_test)

CPU times: user 1.37 ms, sys: 1.21 ms, total: 2.58 ms
Wall time: 3.88 ms


array([[0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [199]:
%time my_clf.predict_proba(X_test)

CPU times: user 2.02 ms, sys: 1.25 ms, total: 3.26 ms
Wall time: 2.13 ms


array([[0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [200]:
clf.score(X_test, y_test)

1.0

In [201]:
my_clf.score(X_test, y_test)

1.0

Задание 3

**MNIST**

In [202]:
mnist = fetch_mldata('MNIST original')

/Users/pavelermolaev/miniconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function fetch_mldata is deprecated; fetch_mldata was deprecated in version 0.20 and will be removed in version 0.22
  warnings.warn(msg, category=DeprecationWarning)
/Users/pavelermolaev/miniconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function mldata_filename is deprecated; mldata_filename was deprecated in version 0.20 and will be removed in version 0.22
  warnings.warn(msg, category=DeprecationWarning)


In [203]:
X_train, X_test, y_train, y_test = train_test_split(mnist.data, mnist.target, test_size=0.01, stratify=mnist.target)

In [204]:
clf = KNeighborsClassifier(n_neighbors=2)
my_clf = MyKNeighborsClassifier(n_neighbors=2)

In [205]:
%time clf.fit(X_train, y_train)

CPU times: user 29.3 s, sys: 212 ms, total: 29.5 s
Wall time: 29.5 s


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=2, p=2,
           weights='uniform')

In [206]:
%time my_clf.fit(X_train, y_train)

CPU times: user 6.06 ms, sys: 123 µs, total: 6.18 ms
Wall time: 6.35 ms


In [ ]:
%time clf.predict(X_test)

In [ ]:
%time my_clf.predict(X_test)

In [ ]:
%time clf.predict_proba(X_test)

In [ ]:
%time my_clf.predict_proba(X_test)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
my_clf.score(X_test, y_test)

Задание 4

In [ ]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(mnist.data, mnist.target, test_size=0.01, stratify=mnist.target)

In [ ]:
%time clf.fit(X_train, y_train)

In [ ]:
%time my_clf.fit(X_train, y_train)

In [ ]:
%time clf.predict(X_test)

In [ ]:
%time my_clf.predict(X_test)

In [ ]:
%time clf.predict_proba(X_test)

In [ ]:
%time my_clf.predict_proba(X_test)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
my_clf.score(X_test, y_test)

Задание 5

In [ ]:
# seaborn
# matplotlib
# train_test_split
# Pipelin%load_ext e (from sklearn.pipeline import Pipeline)
# StandardScaler (from sklearn.preprocessing import StandardScaler)
# ClassifierMixin
# NeighborsBase
# KNeighborsMixin
# SupervisedIntegerMixin